## Reproduce solvent segregation in Wang 2022 - Fig. 4
### $ D_{\text{e},\text{EC}}^0 = 0, \, 2.5 \times 10^{-12}, \, 2.5 \times 10^{-11} \, \text{m}^2/\text{s} $

In [7]:
# magic lines that avoid re-start
%load_ext autoreload
%autoreload 2
import pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp,sqrt;
import matplotlib as mpl; 
fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pybamm.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip/Rio_Code/Fun_P3"))
sys.path.append(str_path_1) 
from Fun_P3 import *
BasicPath = os.path.expanduser("~/EnvPBGEM_ECDrag2/Double_SimSave/Double_Trans/")
Target  = 'Rate_Performance/' 
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target)
Save_Fig = True

In [9]:
# define parameter of double and single 
Para_dict_Same = {
   "Mesh list":[ [20,10,20,100,20], ],   # Simon uses 30
   "Para_Set":[ "Li2023_ECdrag",],
   "Contact resistance [Ohm]":[12e-3],
   "Initial Neg SOC":[0.8841301667966484,],
   "Initial Pos SOC":[0.23552755074598045,],}
Para_dict_DD_ONLY = {
   "Model option":[{
      "SEI": "constant",
      "SEI film resistance":"distributed",   
      "SEI porosity change": "true",
      "solvent diffusion": "double spatial consume w refill",
      "electrolyte conductivity": "sol full"  ,
     "contact resistance":"true",  },],
   "Lithium ion EC cross diffusivity [m2.s-1]":[0,2.5E-12,2.5e-11,], # 
}
Para_dict_SD_ONLY = {
   "Model option":[{
      "SEI": "constant",
      "SEI film resistance":"distributed",   
      "SEI porosity change": "true",
      "solvent diffusion": "single no consume wo refill",
      "electrolyte conductivity": "full"  ,
      "contact resistance":"true",  },],
   "Lithium ion EC cross diffusivity [m2.s-1]":[0,], #  
   "Measured dLJP_dce":["dLJP_1_Specie_dc_e_np",]
}
Para_dict_DD = {**Para_dict_Same,**Para_dict_DD_ONLY}
Para_dict_SD = {**Para_dict_Same,**Para_dict_SD_ONLY}
Para_DD = [];  Para_SD=[];
recursive_scan(Para_DD,Para_dict_DD, list(Para_dict_DD.keys()), {})
recursive_scan(Para_SD,Para_dict_SD, list(Para_dict_SD.keys()), {})
print(len(Para_DD),len(Para_SD))

Path_pack = [BasicPath,Target,]

3 1


#### 2.5e-11 + 2.6C takes 40+ minutes and has spikes; 1e-11 + 2.8C takes 44 minutes, -> both not good

In [10]:
Rate_Dis_All = [1,1.5,2,2.5,2.525,2.55,2.575,2.6]     # np.arange(1,2.5+0.5,0.5) # np.arange(3.8,4.5+0.175,0.175)
print(Rate_Dis_All,len(Rate_Dis_All))

[1, 1.5, 2, 2.5, 2.525, 2.55, 2.575, 2.6] 8


In [11]:
for i in range(len(Para_DD)):  
    for Rate_Dis in Rate_Dis_All:
        D_e_EC_cross = Para_DD[i]["Lithium ion EC cross diffusivity [m2.s-1]"]
        str_pickle =  f"Double_D_e_EC_cross={D_e_EC_cross}"
        sol_return = Run_P3_OneCycle_Dict(
            i, Para_DD[i], Path_pack,Rate_Dis,str_pickle,
            Return_sol=False,Save_sol=True)

Start Now! C rate: 1


In [ ]:
for i in range(len(Para_SD)):  
    for Rate_Dis in Rate_Dis_All:
        D_e_EC_cross = Para_SD[i]["Lithium ion EC cross diffusivity [m2.s-1]"]
        str_pickle =  f"Single"
        sol_return = Run_P3_OneCycle_Dict(
            i, Para_DD[i], Path_pack,Rate_Dis,str_pickle,
            Return_sol=False,Save_sol=True)